## Image Processing with Radar Reflectivity
This program accepts radar scans plotted onto a cartesian grid. The program then thresholds the image to perform image processing and analysis on the largest echo of reflectivity within the image, and outputs a CSV file with the parameters that were calculated for each radar file. </br>

#### Author
<b>Maggie Zoerner</b></br>
Argonne National Laboratory, <i>Environmental Science Division</i></br>
Student Undergraduate Laboratory Intern </br>

#### Import the libraries.

In [1]:
from netCDF4 import Dataset
from scipy import ndimage
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import csv
from glob import glob

#### Import the files.

In [2]:
files = glob('season_grids/KILX2020*')

#### In the below step: </br>

- Go through the data set.
- Remove reflectivities smaller than 35 dBz to remove non-convective precip.
- Clear smaller echoes to remove noise.
- Calculate the raw, central, normalized central, and invariant moments.
- Perform shape computations based on the moments.
- Find the largest external contour of the echo.
- Perform shape computations based on the contours.
- Export computations to CSV file.

In [3]:
# LOOP THROUGH THE DATASET

header_added = False
    
for file in files:
    ncfile = Dataset(file, "r")
    ref = ncfile.variables['reflectivity'][:]
    ref = ref.squeeze()
    ref_thre = np.where(ref < 35, 0, ref)
    labeled_echo = ndimage.label(ref_thre)[0]

    # Remove smaller echoes 
    
    def clear_small_echoes(label_image, min_size):
        """ Takes in binary image and clears objects less than min_size. """
        flat_image = pd.Series(label_image.flatten())
        flat_image = flat_image[flat_image > 0]
        size_table = flat_image.value_counts(sort=False)
        small_objects = size_table.keys()[size_table < min_size]

        for obj in small_objects:
            label_image[label_image == obj] = 0
        label_image = ndimage.label(label_image)
        return label_image[0]
    
    max_echo = clear_small_echoes(labeled_echo, 175)
    
    # Convert to float

    max_echo = max_echo.astype('float32')
    
    # Calculate geometric moments and moment invariants
    
    moments =cv2.moments(max_echo)

    Hu_moments = cv2.HuMoments(moments)
    
    # The 0th moment gives the area
    
    M = moments
    area = M['m00']
    
    # Ensure that the area is not equal to zero (as this will mess up future calculations)
    
    try:
        
        # Calculate the center of mass
        Cx = int(M['m10'] / M['m00'])
        Cy = int(M['m01'] / M['m00'])
        
        # The 3rd normalized central moment gives the skewness (the deviation of the respective projection from symmetry)
    
        skewness_x = (M['nu30'])
        skewness_y = (M['nu03'])

        # Find the deviation from circlular shape
    
        dev_circ = abs(M['nu11'])
    
        # Convert to a format accepted by OpenCV contour analysis
    
        max_echo_u8 = max_echo.astype(np.uint8)
    
        # Find and draw the contours in the image
    
        u8_copy = max_echo_u8.copy()
        contours, hierarchy = cv2.findContours(u8_copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
        cnt = contours[0]
        contour_image = cv2.drawContours(u8_copy, cnt, -1, (0,255,0),3)

        # Find the perimeter of the shape

        perimeter = cv2.arcLength(cnt,True)
    
        # Find the elongation (aspect ratio) of the shape 
    
        x,y,w,h = cv2.boundingRect(cnt)
    
        elongation = float(w)/h

        # Find the equivalent diameter: diameter of a circle whose diameter is the same as the contour area
    
        area = cv2.contourArea(cnt)
        equi_diameter = np.sqrt(4*area/np.pi)
    
        # Find the roundness 
    
        roundness = (4*np.pi*(area))/((perimeter)**2)
    
        # Find the compactness 
    
        compactness = ((perimeter)**2)/(4*np.pi*(area))
    
        # Find the eccentricity 
    
        (x,y),(MA,ma),angle = cv2.fitEllipse(cnt)
        ecc = ma/MA
        

        # WRITING TO A CSV FILE
    
        # Define header names
    
        headers = ['File Name', 'Area', 'Perimeter', 'Center of Mass (X)', 'Center of Mass (Y)', 'Skewness (X)', 'Skewness (Y)', 
                   'Elongation','Equivalent Diameter','Compactness', 'Roundness', 'Eccentricity', 'Deviation from Circle']
 
        # Define data to include
    
        rows = [(file), (area), (perimeter), (Cx), (Cy), (skewness_x), (skewness_y), (elongation), (equi_diameter), 
                (compactness), (roundness), (ecc), (dev_circ)]
 
        # Name the CSV file
    
        filename = "parameter_computations.csv"
    
        # Ensure header does not duplicate in loop and populate file
        
        with open(filename, 'a') as csvfile:
            csvwriter = csv.writer(csvfile)
            if not header_added:
                csvwriter.writerow(headers)
                header_added = True
            csvwriter.writerow(rows)
        
    except ZeroDivisionError:
        pass